## I 202 Implementation Project: Skincare Recommendation Engine

## Data Cleaning

In [1]:
import pandas as pd
import numpy as np
import re

import os
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
from PIL import Image
import io
import requests
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.webdriver.common.by import By

In [89]:
sephora_data = pd.read_csv("data.csv")
sephora_data.head()

/Users/marissakhoury/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Label,URL,brand,description,full_description,name,num_reviews,...,yellow 5 (Ci 19140),Red 28 Lake (CI 45410),Red 7 Lake (CI 15850),Yellow 5 Lake (CI 19140),ingredients,Combination,Dry,Normal,Oily,Sensitive
0,0,0,0,Moisturizers,https://www.sephora.com/product/ultra-facial-c...,Kiehl's Since 1851,A bestselling face moisturizer that’s free of ...,NaN,Ultra Facial Cream,1132.0,...,0,0,0,0,"['Glacial Glycoprotein', 'Squalene', 'Aqua/Wat...",1,1,1,0,0
1,1,1,1,BB & CC Creams,https://www.sephora.com/product/cicapair-tiger...,Dr. Jart+,A green-to-beige color-correcting treatment th...,NaN,Cicapair™ Tiger Grass Color Correcting Treatme...,3065.0,...,0,0,0,0,"['Centella Asiatica Complex', 'Herbs Complex',...",1,1,1,1,0
2,2,2,2,Moisturizers,https://www.sephora.com/product/protini-tm-pol...,Drunk Elephant,A protein moisturizer that combines signal pep...,NaN,Protini™ Polypeptide Moisturizer,4872.0,...,0,0,0,0,"['Signal Peptide Complex (Growth Factors)', 'P...",1,1,1,1,0
3,3,3,3,Moisturizers,https://www.sephora.com/product/creme-de-la-me...,La Mer,A luxuriously rich cream that thoroughly sooth...,NaN,Crème de la Mer Moisturizer,695.0,...,0,0,0,0,"['Algae (Seaweed) Extract', 'Mineral Oil', 'Pe...",0,1,0,0,0
4,4,4,4,Moisturizers,https://www.sephora.com/product/the-dewy-skin-...,Tatcha,A rich cream that feeds skin with plumping hyd...,NaN,The Dewy Skin Cream Plumping & Hydrating Moist...,1727.0,...,0,0,0,0,"['Aqua/Water/Eau', 'Glycerin', 'Propanediol', ...",0,1,1,0,0


In [90]:
# find unnamed columns
for col in sephora_data.columns:
    if col.lower().startswith("unnamed"):
        print(col)
# remove unnamed columns
sephora_data = sephora_data.drop(["Unnamed: 0", "Unnamed: 0.1", "Unnamed: 0.1.1", "Unnamed: 12"], axis=1)

Unnamed: 0
Unnamed: 0.1
Unnamed: 0.1.1
Unnamed: 12


Since Sephora's website lists products from multiple sellers, ingredient names are not normalized (spelling errors, etc.). 

Many ingredients only show up in one product out of 1000+ and are mispelled or different versions of the same product. Here, I remove the ingredients which are present in only one product.

In [99]:
l = sephora_data.sum(axis=0)
new_cols = ['Label', 'URL', 'brand', 'description', 'full_description', 'name',
        'num_reviews', 'price', 'prod_size', 'rating', 'skin_type', 'ingredients',
        'Combination', 'Dry', 'Normal', 'Oily', 'Sensitive']

new_ingredients = []

# Look at integer columns (one-hot-encoded ingredient columns)
# See which one-hot-encoded ingredient column has the most data
columns = l[l.apply(lambda x: isinstance(x, (int, np.integer)))].sort_values(ascending=False)

f = False # flag
for i in columns.index:
    # First real ingredient, others are numeric information columns
    if i == 'Butylene Glycol' or f:
        f = True
        new_ingredients.append(i)
    # "Pentylene Glycol / Ethylhexylglycerin" is the last ingredient with more than 1 occurance
    if i == "Pentylene Glycol / Ethylhexylglycerin": # last 
        break

In [98]:
sephora_data[new_cols].head()

,Label,URL,brand,description,full_description,name,num_reviews,price,prod_size,rating,skin_type,ingredients,Combination,Dry,Normal,Oily,Sensitive
0,Moisturizers,https://www.sephora.com/product/ultra-facial-c...,Kiehl's Since 1851,A bestselling face moisturizer that’s free of ...,NaN,Ultra Facial Cream,1132.0,32.0,1.70,4.3,"['Dry', 'Normal', 'Combination']","['Glacial Glycoprotein', 'Squalene', 'Aqua/Wat...",1,1,1,0,0
1,BB & CC Creams,https://www.sephora.com/product/cicapair-tiger...,Dr. Jart+,A green-to-beige color-correcting treatment th...,NaN,Cicapair™ Tiger Grass Color Correcting Treatme...,3065.0,52.0,1.70,4.1,"['Normal', 'Dry', 'Combination', 'Oily']","['Centella Asiatica Complex', 'Herbs Complex',...",1,1,1,1,0
2,Moisturizers,https://www.sephora.com/product/protini-tm-pol...,Drunk Elephant,A protein moisturizer that combines signal pep...,NaN,Protini™ Polypeptide Moisturizer,4872.0,68.0,1.69,4.0,"['Normal', 'Dry', 'Combination', 'Oily']","['Signal Peptide Complex (Growth Factors)', 'P...",1,1,1,1,0
3,Moisturizers,https://www.sephora.com/product/creme-de-la-me...,La Mer,A luxuriously rich cream that thoroughly sooth...,NaN,Crème de la Mer Moisturizer,695.0,190.0,1.00,3.9,['Dry'],"['Algae (Seaweed) Extract', 'Mineral Oil', 'Pe...",0,1,0,0,0
4,Moisturizers,https://www.sephora.com/product/the-dewy-skin-...,Tatcha,A rich cream that feeds skin with plumping hyd...,NaN,The Dewy Skin Cream Plumping & Hydrating Moist...,1727.0,68.0,1.70,4.2,"['Normal', 'Dry']","['Aqua/Water/Eau', 'Glycerin', 'Propanediol', ...",0,1,1,0,0


In [46]:
# get all the ingredients columns
with open('ingredients.txt', 'w') as file:
    for col in sephora_data.columns:
        file.write(col+"\n")

In [105]:
new_ingredients

['Butylene Glycol',
 'Retinyl Palmitate',
 'Ethylhexylglycerin',
 'Tocopherol',
 'Mineral Oil',
 'Phthalates',
 'Carbon Black',
 'Formaldehydes',
 'Ethanolamines DEA/TEA/MEA/ETA',
 'methyl methacrylate',
 'Deastyrene/acrylates/dvbcopolymer',
 'Coal Tar',
 'hydroxypropyl methacrylate',
 'Formaldehyde-releasing agents',
 'ethyl methacrylate',
 'Toluene',
 'Triclocarban',
 'Resorcinol',
 'butyl methacrylate',
 'Butylated hydroxyanisole (BHA)',
 'Butoxyethanol',
 'aluminum salts)',
 'Benzophenone + Related Compounds',
 'Triclosan',
 'Animal Oils/Musks/Fats',
 'Hydroquinone',
 'Methyl Cellosolve + Methoxyethanol',
 'Polyacrylamide & Acrylamide',
 'Mercury + Mercury Compounds (Thimerisol)',
 'styrene oxide',
 'Lead/Lead Acetate',
 'styrene)',
 'Methylchloroisothiazolinone & Methylisothiazolinone',
 'tetrahydrofurfuryl methacrylate',
 'trimethylolpropane trimethacrylate',
 'Oxybenzone',
 'Parabens',
 'sodium styrene/divinylbenzene copolymer',
 'Sodium Hyaluronate',
 'Xanthan Gum',
 'Citric Ac

## Scrape EWG Ingredient Data

https://www.ewg.org/skindeep/

In [135]:
# scraping ingredient info from EWG Skin Deep
# want a function that creates a normalized ingredient
import urllib
import os
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
from PIL import Image
import io
import requests
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.webdriver.common.keys import Keys

#from org.openqa.selenium import Keys

UBLOCK_ORIGIN_EXTENSION_PATH = "/Users/marissakhoury/Library/Application Support/Google/Chrome/Default/Extensions/cjpalhdlnbpafiamejdnhcphjbkeiagm/1.39.2_9"
CHROME_PATH = "driver/chromedriver"
# setting up the options for chrome driver, loading extension
chrome_options = Options()
chrome_options.add_argument('load-extension=' + UBLOCK_ORIGIN_EXTENSION_PATH)
chrome_options.add_argument('chrome')
chrome_options.add_argument("--disable-popup-blocking");

In [102]:
# step 1: spin up driver
driver = webdriver.Chrome(CHROME_PATH, chrome_options=chrome_options)
driver.create_options()
time.sleep(10)
# last step: close it
driver.close()

/Users/marissakhoury/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: use options instead of chrome_options
  


In [142]:
# Helpers

failed_to_parse = []
def lookup_ingredient(ingredient_name):
    # step 1: set up
    chrome_options = Options()
    chrome_options.add_argument('load-extension=' + UBLOCK_ORIGIN_EXTENSION_PATH)
    chrome_options.add_argument('chrome')
    chrome_options.add_argument("--enable-popup-blocking");
    driver = webdriver.Chrome(CHROME_PATH, chrome_options=chrome_options)
    driver.create_options()
    try:
        
        home_page = 'https://www.ewg.org/skindeep'
        output = [] # store the data here as a row
        
        
        search_result_page = 'https://www.ewg.org/skindeep/search/?search={}&search_type=ingredients'.format(
            urllib.parse.quote_plus(ingredient_name)
        )
        driver.get(search_result_page)# navigate to page
        
        
        time.sleep(0.5)
#         search_bar_id = "search"
#         s = driver.find_element_by_id(search_bar_id)
#         s.send_keys(ingredient_name)
#         s.send_keys(Keys.RETURN)

        # parse results
        elems = driver.find_elements_by_class_name('product-tile')
#         for elem in elems:
#             print(elem.get_attribute('innerHTML'))
#             print('---')

        # first search result
        m = re.search(r"""href=\"(.*?)\"""", elems[0].get_attribute("innerHTML"))
        if not m:
            return None

        product_url = 'https://www.ewg.org' + m.group(1)
        driver.get(product_url)
        # parse for info
        ingredient = driver.find_element_by_id("chemical")
#         print("ingredient", ingredient)
#         print(ingredient.get_attribute('innerHTML'))

        # chemical name
        chemical_name_h2 = ingredient.find_element_by_class_name("chemical-name")
#         print(chemical_name_h2.text)
#         print(chemical_name_h2.text)

        # score
        score = driver.find_element_by_class_name("squircle")
#         print("score", score.get_attribute("src"))
        
        m = re.search(r'score=([0-9]+)', score.get_attribute("src"))
        if not m:
            return None
        print('Score for {} = {}'.format(ingredient_name, m.group(1)))
        return m.group(1)

    #     # see all chemical
    #     chemical_info = driver.find_element_by_class_name("see-all-chemical")
    #     print("chemical info", chemical_info)
    #     print(chemical_info.get_attribute('innerHTML'))

    #     # about chemical
    #     about_chemical_pattern = r""""chemical-info chemical-functions-text hidden">(.*)</p>"""
    #     about_match = re.search(about_chemical_pattern, chemical_info.get_attribute('innerHTML'))
    #     print(about_match.group(1))
    # #     print(score.get_attribute('innerHTML'))
    except Exception as e:
        print('{}-FAILED to parse:'.format(ingredient_name), e)
        
    finally:
        driver.close()
    
# lookup_ingredient("Butylene Glycol")

In [143]:
# do it for all ingredients
ingredient_scores = {}
for ing in new_ingredients:
    ingredient_scores[ing] = lookup_ingredient(ing)

/Users/marissakhoury/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: DeprecationWarning: use options instead of chrome_options


Score for Butylene Glycol = 1
Score for Retinyl Palmitate = 9
Score for Ethylhexylglycerin = 2
Score for Tocopherol = 1
Score for Mineral Oil = 1
Score for Phthalates = 7
Score for Carbon Black = 1
Score for Formaldehydes = 6
Score for Ethanolamines DEA/TEA/MEA/ETA = 4
Score for methyl methacrylate = 1
Deastyrene/acrylates/dvbcopolymer-FAILED to parse: list index out of range
Score for Coal Tar = 3
Score for hydroxypropyl methacrylate = 1
Formaldehyde-releasing agents-FAILED to parse: list index out of range
Score for ethyl methacrylate = 1
Score for Toluene = 3
Score for Triclocarban = 3
Score for Resorcinol = 4
Score for butyl methacrylate = 1
Score for Butylated hydroxyanisole (BHA) = 6
Score for Butoxyethanol = 5
Score for aluminum salts) = 2
Benzophenone + Related Compounds-FAILED to parse: list index out of range
Score for Triclosan = 1
Animal Oils/Musks/Fats-FAILED to parse: list index out of range
Score for Hydroquinone = 7
Methyl Cellosolve + Methoxyethanol-FAILED to parse: li

Score for Ammonium Acryloyldimethyltaurate/Vp Copolymer = 1
Score for Disteardimonium Hectorite = 1
Score for Parfum/Fragrance = 8
Score for Benzyl Salicylate = 5
Score for Eucalyptus Globulus Leaf Oil = 2
Score for Alumina = 1
Score for Shea Butter = 1
Score for Panax Ginseng Root Extract = 1
Score for Triethanolamine = 5
Score for Pentaerythrityl Tetraethylhexanoate = 1
Score for Sorbitan Stearate = 3
Score for Polyacrylate Crosspolymer-6 = 1
Score for Copper Gluconate = 2
Score for Caramel = 2
Score for Citrus Limon (Lemon) Peel Oil = 6
Score for Dimethyl Isosorbide = 1
Score for Oryza Sativa (Rice) Bran Extract = 2
Score for Helianthus Annuus (Sunflower) Seedcake = 1
Score for Sclerocarya Birrea Seed Oil = 1
Score for Vitamin E = 1
Score for Iron Oxides (Ci 77492) = 2
Score for Trisodium Edta = 1
Score for Ammonium Acryloyldimethyltaurate/VP Copolymer = 1
Score for Chlorella Vulgaris Extract = 1
Score for Alanine = 1
Score for Sodium Pca = 6
Score for Calcium Gluconate = 1
Score fo

Yeast Extract\Faex\Extrait De Levure-FAILED to parse: list index out of range
Score for Macadamia Ternifolia Seed Oil = 1
Score for Magnesium Gluconate = 1
Score for Neopentyl Glycol Diheptanoate = 1
Score for Argania Spinosa Kernel Oil = 1
Score for Tocopheryl Succinate = 1
Score for Glycine Soja (Soybean) Protein = 1
Score for Melia Azadirachta Flower Extract = 1
Score for Polyquaternium-51 = 4
Score for Medicago Sativa (Alfalfa) Seed Powder = 1
Score for Hydroxypropyl Methylcellulose = 1
Score for Vitis Vinifera (Grape) Seed Extract = 1
Score for Isopropyl Palmitate = 1
Score for Charcoal Powder = 1
Score for Hydrolyzed Soy Protein = 1
Score for Diglycerin = 2
Score for Hydroxyproline = 1
Score for Tridecyl Trimellitate = 1
Score for Trimethylsiloxysilicate = 1
Score for Saccharomyces Lysate Extract = 1
Score for Diethylhexyl Syringylidenemalonate = 1
Score for Octisalate 5% = 3
Score for CI 77891 (Titanium Dioxide) = 3
Hordeum Vulgare (Barley) Extract/Extrait D'Orge-FAILED to parse

MaxRetryError: HTTPConnectionPool(host='127.0.0.1', port=53484): Max retries exceeded with url: /session/075d5d26651dc968e3fed110168ff6d9/window (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fe524500cd0>: Failed to establish a new connection: [Errno 61] Connection refused'))

In [152]:
missing_ingredients = []
for ing in ingredient_scores:
    if ingredient_scores[ing] == None:
        missing_ingredients.append(ing)
missing_ingredients

['Deastyrene/acrylates/dvbcopolymer',
 'Formaldehyde-releasing agents',
 'Benzophenone + Related Compounds',
 'Animal Oils/Musks/Fats',
 'Methyl Cellosolve + Methoxyethanol',
 'Mercury + Mercury Compounds (Thimerisol)',
 'Sensitive',
 'Citrullus Lanatus (Watermelon) Fruit Extract',
 'Yeast Extract/Faex/Extrait De Levure',
 'Yeast Extract\\Faex\\Extrait De Levure',
 "Hordeum Vulgare (Barley) Extract/Extrait D'Orge",
 'Whey Protein/Lactis Protein/Proteine Du Petit-Lait',
 'Tasmannia Lanceolata Fruit/Leaf Extract',
 'Agastache Mexicana Flower/Leaf/Stem Extract',
 'Tropaeolum Majus Flower/Leaf/Stem Extract',
 'Natural Fragrance',
 'Tetrahexyldecyl Ascorbate (Vitamin C)',
 'Cocos Nucifera (Coconut) Fruit Extract',
 'Hydroxypinacolone Retinoate',
 'Phytosteryl/Isostearyl/Cetyl/Stearyl/Behenyl Dimer Dilinoleate',
 'Hydro Ionized Mineral Water',
 'Aristotelia Chilensis (Maqui) Fruit Extract',
 'Berry Mix Complex™',
 'Green Fusion Complex™',
 'Gellidiela Acerosa (Algae) Extract',
 'Fragrance (N

In [155]:
sephora_data[new_cols + new_ingredients].head().columns

Index(['Label', 'URL', 'brand', 'description', 'full_description', 'name',
       'num_reviews', 'price', 'prod_size', 'rating',
       ...
       'Limonium Vulgare Flower/Leaf/Stem Extract', 'Limonium Gerberi Extract',
       'Lycopene', 'Limolool', 'Limnanthes alba (Meadowfoam) Seed Oil',
       'Polyglyceryl-3 Rice Branate', 'Lime Tea Concentrate',
       'Perfluoromethylcyclopentane', 'Oat Sugars',
       'Pentylene Glycol / Ethylhexylglycerin'],
      dtype='object', length=2965)

In [147]:
i = 0
starting_point = 'Sodium Polyglutamate'
startFlag = False
for ing in new_ingredients:
    if startFlag or ing == starting_point:
        print(i)
        ingredient_scores[ing] = lookup_ingredient(ing)
        startFlag = True
    i += 1

1096


/Users/marissakhoury/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: DeprecationWarning: use options instead of chrome_options


Score for Sodium Polyglutamate = 1
1097
Score for Cera Alba/Beeswax = 1
1098
SkingenecellEnmei-FAILED to parse: list index out of range
1099
Score for Sleep-Tox™ = 1
1100
Tatcha's Signature Hadasei-3-FAILED to parse: list index out of range
1101
Score for Dextran = 3
1102
Score for Butylene / Ethylene / Styrene Copolymer = 1
1103
Tasmannia Lanceolata Fruit/leaf Extract-FAILED to parse: list index out of range
1104
Tasmanian Pepperberry-FAILED to parse: list index out of range
1105
Score for Snail Secretion Filtrate = 1
1106
Score for Uji Green Tea Extract = 1
1107
Score for Alpha-isomethyl Ionone = 5
1108
Score for Ceramide NG = 1
1109
Score for Alteromonas Ferment Extract = 1
1110
Score for Angelica Keiskei Leaf/Stem Extract = 1
1111
Score for Ci 77492) = 2
1112
Score for Bupleurum Falcatum Root Extract = 1
1113
Chrysanthemum Indicum Flower Extract-FAILED to parse: list index out of range
1114
Score for Aluminum Starch Octenylsuccinate = 2
1115
Score for Ammonium Acryloyldimethyltaura

Score for Orbignya Oleifera Seed Oil = 1
1271
Score for Ethylhexyl Isononanoate = 1
1272
Score for Rosa Canina Seed Oil = 3
1273
Score for Opuntia Coccinellifera Fruit Extract = 1
1274
Score for Hexylresorcinol = 4
1275
Score for Essential Oil = 3
1276
Score for Paeonia Albiflora Root Extract = 1
1277
Score for Parfum = 8
1278
Score for Palmitoyl Dipeptide-5 Diaminobutyroyl Hydroxythreonine = 1
1279
Score for Honey Extract = 1
1280
Score for Ginseng Extract = 1
1281
Score for Hordeum Vulgare (Barley) Extract = 1
1282
Ocimum Basilicum (Basil) Flower/Leaf Extract-FAILED to parse: list index out of range
1283
Score for Palmitoyl Grapevine Shoot Extract = 1
1284
Score for Diethoxyethyl Succinate = 1
1285
Score for Niacinamide (Vitamin B3) = 1
1286
Score for Boswellia Serrata Extract = 1
1287
Score for Persea Gratissima Oil/Avocado Oil = 1
1288
Score for Parachlorella Beijerinckii Exopolysaccharides = 1
1289
Score for Silybum Marianum Seed Extract = 1
1290
Score for Brassica Campestris (Rap

Score for Decylene Glycol = 1
1442
Score for Sodium Acrylate/Acryloyldimethyltaurate/Dimethylacrylamide Crosspolymer = 1
1443
Score for Dehydroxanthan Gum = 1
1444
Score for Hydrogenated Castor Oil Isostearate = 1
1445
Score for Hydrogenated Castor Oil Dimer Dilinoleate = 1
1446
Score for Denatonium Benzoate = 1
1447
Score for Simmondsia Chinesis (Jojoba) Seed Oil = 1
1448
Score for Red 7 (Ci 15850) = 6
1449
Score for Sodium Glycolate = 1
1450
Score for Sodium Lauroyl Methyl Isethionate = 1
1451
Score for Cocoglycerides = 1
1452
Score for Pyridoxine Dipalmitate = 1
1453
Score for Coconut Acid = 1
1454
Score for Coconut Water = 5
1455
Sorbic Acid. *PITERA™-FAILED to parse: list index out of range
1456
Score for Soluble Collagen = 1
1457
Score for Codium Tomentosum Extract = 1
1458
Score for Coix Lacryma-Jobi Ma-Yuen Seed Extract = 1
1459
Score for Colloidal Gold = 2
1460
Score for Sodium Potassium Aluminum Silicate = 2
1461
Score for Iris Pallida Root Extract = 1
1462
Score for Cotton S

...

Saccharomyces Complex-FAILED to parse: list index out of range
2263
Score for Gamma-Decalactone = 1
2264
Score for Gardenia Jasminoides (Gardenia) Fruit Extract = 1
2265
Score for Saccharin = 1
2266
Score for Saccharide Hydrolysate = 1
2267
Score for Gaultheria Procumbens (Wintergreen) Leaf Oil = 1
2268
STEP 02 Hydro Liquid Ampoule:-FAILED to parse: list index out of range
2269
STEP 01 Super Booster:-FAILED to parse: list index out of range
2270
Score for SD Alcohol 39B = 3
2271
Score for Genistein = 1
2272
Score for Gentiana Lutea Root Extract = 1
2273
Score for Salicylic Acid (BHA) = 3
2274
Flaxseed Oil-FAILED to parse: list index out of range
2275
Score for HA³ = 4
2276
Exclusive Anti-Pollution Complex-FAILED to parse: list index out of range
2277
Score for Eucalyptus Globululs Leaf Oil = 2
2278
Score for Eucalyptus Globulus Leaf = 2
2279
Score for Saponins = 2
2280
Score for Eucalyptus Globulus Leaf Powder = 1
2281
Score for Eugenia Caryophyllus (Clove) Leaf Oil = 4
2282
Eugenia Ca

Score for Bis-Diglyceryl Polyacyladipate-1 = 1
2429
Score for Bis-Glyceryl Ascorbate = 1
2430
Score for Bis-Vinyl Dimethicone/Dimethicone Copolymer = 1
2431
Vegan. Cruelty-free. Free of Parabens-FAILED to parse: list index out of range
2432
Score for Bismuth Oxychloride = 1
2433
Black Hijiki Seaweed-FAILED to parse: list index out of range
2434
Score for Black Tea Ferment = 1
2435
Score for Vegan Silk Protein = 1
2436
Score for Blue Tansy Oil = 1
2437
Score for Blue-green Algae = 1
2438
Score for Blueberry Oil = 1
2439
Score for Vanillyl Butyl Ether = 1
2440
Score for Vanilla Tahitensis Fruit Extract = 1
2441
Born in Bordeaux-FAILED to parse: list index out of range
2442
Score for Boswellia Carterii (Olibanum) Oil = 1
2443
Botanical Eye Contour Complex-FAILED to parse: list index out of range
2444
Score for Viola Mandshurica Flower Extract = 1
2445
Score for Beta-Carotene (CI 40800) = 1
2446
Score for Baccharis Genistelloides Extract = 1
2447
Virgin Coconut Oil-FAILED to parse: list in

Score for Cha-7 es complex = 4
2596
Score for Chamomil = 2
2597
Score for Chamomila Recutita (Matricaria) Flower Oil = 2
2598
Score for Chamomilla Recutita (Matricaria) Flower Water = 2
2599
Score for Chamomilla Recutita Flower Extract = 2
2600
Score for Charcoal = 1
2601
TEAOXI® Eucalyptus Leaf-FAILED to parse: list index out of range
2602
Score for Chia Seed Oi = 1
2603
T.L.C. Framboos™ AHA Blend-FAILED to parse: list index out of range
2604
Score for Chitosan Ascorbate = 1
2605
Chlolleragen-FAILED to parse: list index out of range
2606
Chlorella Protothecoides-FAILED to parse: list index out of range
2607
Score for Valeriana Officinalis Root Oil = 1
2608
Score for Synthetic Dyes = 2
2609
Symphytum Officinale Callus Culture Extract-FAILED to parse: list index out of range
2610
Score for Cholesteryl Isostearate = 1
2611
Score for Swertia Chirata Extract = 1
2612
Surgical Grade Pure White Petrolatu-FAILED to parse: list index out of range
2613
Chondrus Crispus (Red Algae) Extract-FAILE

Score for Undecylenoyl Phenylalanine = 1
2759
Score for Butyrospermum Parkii Butter (Shea Butter) = 1
2760
Score for Undecane = 1
2761
Score for Undaria Pinnatiﬁda Extract = 1
2762
Score for Ultramarines (Ci 77007) = 3
2763
Score for C12-20 Alkyl Gucoside = 1
2764
Score for C30-38 Olefin/Isopropyl Maleate/Ma Copolymer = 1
2765
Score for Calcium Pca = 2
2766
Score for Trideceth-6 = 1
2767
Score for Thermus Thermophillus Fer -Ment = 1
2768
Score for Titanium Dioxide [Nano] / Titanium Dioxide = 3
2769
Score for Titanium Dioxide 1.70%Water = 3
2770
Score for Caprylyl Trimethicone = 1
2771
Score for Titanium Dioxide 0.95% = 3
2772
Score for Capsicum Annuum Fruit Extract = 1
2773
Score for Caramel Color = 1
2774
Score for Carbon = 4
2775
Score for Tin Oxide (CI 77861) = 3
2776
Score for Carica Papaya (Papaya) Fruit = 1
2777
Score for Carica Papaya Fruit Extract = 1
2778
Score for Carnauba Wax = 1
2779
Tiger Grass (Pennywort)-FAILED to parse: list index out of range
2780
Thymuscitriodorus Flo

Potent Deep Ocean Red Algae-FAILED to parse: list index out of range
2927
Potent Silk Tree Bark Blend-FAILED to parse: list index out of range
2928
Score for Peg-30 Dipolyhydroxystearate = 3
2929
Lactic Acid (AHA) from Yeast-FAILED to parse: list index out of range
2930
Score for P-Anisic Acid = 1
2931
Phenoxyethanol. *Essential Oil-FAILED to parse: list index out of range
2932
Score for Polydimethylsiloxane/Polymethylsilsesquioxane Copolymer = 1
2933
Score for Lavandula Angustifolia (Lavender) Flower Water = 1
2934
Score for Kahai Oil = 1
2935
Score for Oat Kernel Extract = 1
2936
Linalool* (182/035)-FAILED to parse: list index out of range
2937
Score for Polygonatum Officinale Rhizome/Root Extract = 1
2938
Limonium Vulgare Flower/Leaf/Stem Extract-FAILED to parse: list index out of range
2939
Score for Limonium Gerberi Extract = 1
2940
Score for Lycopene = 1
2941
Score for Limolool = 3
2942
Score for Limnanthes alba (Meadowfoam) Seed Oil = 1
2943
Polyglyceryl-3 Rice Branate-FAILED to

In [150]:
len(ingredient_scores.keys())

import csv

# open file for writing, "w" is writing
w = csv.writer(open("ingredient_scores.csv", "w"))

# loop over dictionary keys and values
for key, val in ingredient_scores.items():

    # write every key and value to file
    w.writerow([key, val])

In [ ]:
# testing the data scrape
EWG_URL = "https://www.ewg.org/skindeep/"

# step 1: set up
driver = webdriver.Chrome(CHROME_PATH, chrome_options=chrome_options)
chrome_options = Options()
chrome_options.add_argument('load-extension=' + UBLOCK_ORIGIN_EXTENSION_PATH)
chrome_options.add_argument('chrome')
chrome_options.add_argument("--enable-popup-blocking");
driver.create_options()

# nav to page
driver.get(EWG_URL)

# search for ingredient
search_bar_id = "search"
s = driver.find_element_by_id(search_bar_id)
s.send_keys("Grape Seed Polyphenols")
s.send_keys(Keys.RETURN)

first_result_xpath = '/html/body/div[1]/div[1]/main/section[4]/div'
result = driver.find_element_by_xpath(first_result_xpath)

link = driver.find_element_by_xpath("/html/body/div[1]/div[1]/main/section[4]/div/a")
print(link)
link.click()
# result.click()


# parse for info

time.sleep(20)
driver.close()

In [22]:
import ast
ast.literal_eval(sephora_data.ingredients[0])

['Glacial Glycoprotein',
 'Squalene',
 'Aqua/Water',
 'Glycerin',
 'Cyclohexasiloxane',
 'Squalane',
 'Bis-PEG-18 Methyl Ether Dimethyl Silane',
 'Sucrose Stearate',
 'Stearyl Alcohol',
 'PEG-8 Stearate',
 'Myristyl Myristate',
 'Pentaerythrityl Tetraethlhexanoate',
 'Prunus Armeniaca Kernal Oil/Apricot Kernal Oil',
 'Phenoxyethanol',
 'Persea Gratissima Oil/Avocado Oil',
 'Glyceryl Stearate',
 'Cetl Alcohol',
 'Oryza Sativa Bran Oil/Rice Bran Oil',
 'Olea Europaea Fruit Oil/G4Olive Fruit Oil',
 'Chlorphenesin',
 'Stearic Acid',
 'Palmitic Acid',
 'Disodium EDTA',
 'Acrylates/C10-30 Alkyl Acrylate Crosspolymer',
 'Carbomer',
 'Xanthan Gum',
 'Ethylhexylglycerin',
 'Sodium Hydroxide',
 'Tocopherol',
 'Glycine Soja Oil/Soybean Oil',
 'Pseudoalteromonas Ferment Extract',
 'Myristic acid',
 'Hydroxypalmitol Sphinganine',
 'BHT',
 'Salicylic Acid',
 'Citric Acid']

In [24]:
described = sephora_data.describe()

In [78]:
l = sephora_data.sum(axis=0)

columns = l[l.apply(lambda x: isinstance(x, (int, np.integer)))].sort_values(ascending=False)


f = False

new_cols = ['Label', 'URL', 'brand', 'description', 'full_description', 'name',
       'num_reviews', 'price', 'prod_size', 'rating', 'skin_type']
for i in columns.index:
    if i == 'Butylene Glycol' or f:
        f = True
        new_cols.append(i)    
    
    if i == "Pentylene Glycol / Ethylhexylglycerin":
        break

Butylene Glycol 499
Retinyl Palmitate 429
Ethylhexylglycerin 421
Tocopherol 414
Mineral Oil 387
Phthalates 385
Methyl Cellosolve + Methoxyethanol 383
Formaldehyde-releasing agents 383
Formaldehydes 383
Hydroquinone 383
Animal Oils/Musks/Fats 383
Lead/Lead Acetate 383
Mercury + Mercury Compounds (Thimerisol) 383
methyl methacrylate 383
Methylchloroisothiazolinone & Methylisothiazolinone 383
hydroxypropyl methacrylate 383
Ethanolamines DEA/TEA/MEA/ETA 383
Oxybenzone 383
Parabens 383
Polyacrylamide & Acrylamide 383
ethyl methacrylate 383
Resorcinol 383
butyl methacrylate 383
aluminum salts) 383
Toluene 383
Triclosan 383
styrene oxide 383
Triclocarban 383
sodium styrene/divinylbenzene copolymer 383
Butylated hydroxyanisole (BHA) 383
tetrahydrofurfuryl methacrylate 383
Butoxyethanol 383
Carbon Black 383
styrene) 383
trimethylolpropane trimethacrylate 383
Benzophenone + Related Compounds 383
Coal Tar 383
Deastyrene/acrylates/dvbcopolymer 383
Sodium Hyaluronate 371
Xanthan Gum 361
Citric Acid

Snail Secretion Filtrate 6
Oryza Sativa (Rice) Powder 6
Tasmannia Lanceolata Fruit/leaf Extract 6
Gellan Gum 6
Thioctic Acid 6
Tasmanian Pepperberry 6
Butylene / Ethylene / Styrene Copolymer 6
Isodonis Japonicus Leaf/Stalk Extract 6
Ci 77492) 6
Angelica Keiskei Leaf/Stem Extract 6
Eugenia Caryophyllus (Clove) Flower Extract 6
Sodium hyaluronate 6
CI 77499) 6
Opuntia Ficus-Indica Extract 6
PPG-5-Ceteth-20 6
Ziziphus Jujuba Fruit Extract 6
Triisodecyl Trimellitate 6
Octoxynol-9 6
Uji Green Tea Extract 6
Glycosaminoglycans 6
Sclerocarya Birrea (Marula) Seed Oil 6
Artemisia Princeps Leaf Extract 6
Cannabis Sativa (Hemp) Seed Oil 6
Sunflower Seed Oil 6
Step 1: 6
Chrysanthemum Indicum Flower Extract 6
Aluminum Starch Octenylsuccinate 6
Polyglyceryl-10 Pentastearate 6
Arginine/Lysine Polypeptide 6
Polyglyceryl-6 Distearate 6
C12-20 Alkyl Glucoside 6
Artemia Extract 6
Alteromonas Ferment Extract 6
Lysolecithin 6
Alpha-isomethyl Ionone 6
Bis-PEG-18 Methyl Ether Dimethyl Silane 6
Caprylic /Capri

Gylcerin 3
Styrax Tonkinensis Resin Extract 3
Potassium Alum 3
Rosemary Leaf Extract 3
Cyanocobalamin 3
Acetyl Glutamine 3
PVM/MA Decadiene Crosspolymer 3
Super Lotus 3
Padina Pavonica Thallus Extract 3
Copper Pca 3
Hexyl Nicotinate 3
Potassium Methoxysalicylate 3
Lactobacillus Ferment Lysate 3
Sunflower Extract 3
Palmitoyl Carnitine 3
Citrus Junos Fruit Extract 3
Lactic Acid/Glycolic Acid Copolymer 3
Hydrogenated Styrene/Isoprene Copolymer 3
LGBTQ inclusivity 3
C18-38 Alkyl Hydroxystearoyl Stearate 3
Sucrose Palmitate 3
Sucrose Distearate 3
Chromium Oxide Greens (Ci 77288) 3
Sucrose Cocoate 3
Fragaria Vesca (Strawberry) Fruit Extract 3
Rosemary Essential Oil 3
Pan-Thenol 3
Purified Water 3
Chronocyclin 3
Caprylic/Capric Glycerides 3
Curcuma Longa Extract/Turmeric Root Extract 3
Tetraacetylphytosphingosine 3
Tricaprylin 3
Cetyl Lactate 3
Cucumis Sativus (Cucumber) Oil 3
Epilobium Angustifolium Flower\Leaf\Stem Extract 3
Triacetin 3
Ethylhexyglycerin 3
Tremella Mushroom 3
Prunus Armenia

Bright T-Ferment™ 2
Brazilian White Tourmaline 2
Brazilian Micronized Tourmaline Gemstone Powder 2
Phenoxyethanol *Hadasei-3 2
Phenoxyethanol. *Essential Oil 2
Phenoxyethanol. Acrylates/C10-30 Alkyl Acrylate Crosspolymer 2
Phenyl Methicone 2
Phenyl Trimethicone Glycerin Dipropylene Glycol 2
Activated-X Charcoal 2
Tripeptide 2
Phoenix Dactylifera (Date) Seed Extract 2
Brassica Campestris (Rapeseed) Seed Oil 2
Bottom Layer: 2
Physalis Pubescens Fruit Juice 2
Botanicals 2
Phytoestrogens 2
Botanical Eye Contour Complex 2
Plant Collagen 2
Plant origin 2
Polyglyceryl-¬‐10 Isostearate 2
Alaria Esculenta Extract 2
Polyglyceryl-10 Behenate/Eicosadioate 2
Tocopheryl Linoleate/Oleate 2
Bis-Vinyl Dimethicone/Dimethicone Copolymer 2
Polyglyceryl-10 Oleate 2
Polyglyceryl-2 Isostearate/Dimer Dilinoleate Copolymer 2
Bis-Glyceryl Ascorbate 2
Tocopheryl 2
Polyglyceryl-3 Cocoate 2
Polyglyceryl-3 Polydimethylsiloxyethyl Dimethicone 2
Tocopherol (CI 77891) 2
Polyglyceryl-3 Rice Branate 2
Aden- Osine 2
Bis-

In [86]:
len(new_cols)

3133

In [226]:
import pandas as pd
import csv

ingred_score_dict = {}

with open('ingredient_scores.csv', mode='r') as inp:
    reader = csv.reader(inp)
    ingred_score_dict = {rows[0]:rows[1] for rows in reader}

print(ingred_score_dict)

{'Butylene Glycol': '1', 'Retinyl Palmitate': '9', 'Ethylhexylglycerin': '2', 'Tocopherol': '1', 'Mineral Oil': '1', 'Phthalates': '7', 'Carbon Black': '1', 'Formaldehydes': '6', 'Ethanolamines DEA/TEA/MEA/ETA': '4', 'methyl methacrylate': '1', 'Deastyrene/acrylates/dvbcopolymer': '1', 'Coal Tar': '3', 'hydroxypropyl methacrylate': '1', 'Formaldehyde-releasing agents': '6', 'ethyl methacrylate': '1', 'Toluene': '3', 'Triclocarban': '3', 'Resorcinol': '4', 'butyl methacrylate': '1', 'Butylated hydroxyanisole (BHA)': '6', 'Butoxyethanol': '5', 'aluminum salts)': '2', 'Benzophenone + Related Compounds': '4', 'Triclosan': '1', 'Animal Oils/Musks/Fats': '1', 'Hydroquinone': '7', 'Methyl Cellosolve + Methoxyethanol': '1', 'Polyacrylamide & Acrylamide': '4', 'Mercury + Mercury Compounds (Thimerisol)': '10', 'styrene oxide': '1', 'Lead/Lead Acetate': '1', 'styrene)': '2', 'Methylchloroisothiazolinone & Methylisothiazolinone': '7', 'tetrahydrofurfuryl methacrylate': '5', 'trimethylolpropane tri

In [227]:
scores = ['1', '2', '3', '4', '5', '6', '7','8', '9', '10']
for key, value in ingred_score_dict.items():
    if ingred_score_dict[key] in scores:
        ingred_score_dict[key] = int(value)
    else:
        ingred_score_dict[key] = 0
ingred_score_dict

{'Butylene Glycol': 1,
 'Retinyl Palmitate': 9,
 'Ethylhexylglycerin': 2,
 'Tocopherol': 1,
 'Mineral Oil': 1,
 'Phthalates': 7,
 'Carbon Black': 1,
 'Formaldehydes': 6,
 'Ethanolamines DEA/TEA/MEA/ETA': 4,
 'methyl methacrylate': 1,
 'Deastyrene/acrylates/dvbcopolymer': 1,
 'Coal Tar': 3,
 'hydroxypropyl methacrylate': 1,
 'Formaldehyde-releasing agents': 6,
 'ethyl methacrylate': 1,
 'Toluene': 3,
 'Triclocarban': 3,
 'Resorcinol': 4,
 'butyl methacrylate': 1,
 'Butylated hydroxyanisole (BHA)': 6,
 'Butoxyethanol': 5,
 'aluminum salts)': 2,
 'Benzophenone + Related Compounds': 4,
 'Triclosan': 1,
 'Animal Oils/Musks/Fats': 1,
 'Hydroquinone': 7,
 'Methyl Cellosolve + Methoxyethanol': 1,
 'Polyacrylamide & Acrylamide': 4,
 'Mercury + Mercury Compounds (Thimerisol)': 10,
 'styrene oxide': 1,
 'Lead/Lead Acetate': 1,
 'styrene)': 2,
 'Methylchloroisothiazolinone & Methylisothiazolinone': 7,
 'tetrahydrofurfuryl methacrylate': 5,
 'trimethylolpropane trimethacrylate': 5,
 'Oxybenzone': 4

In [235]:
# score of 0 means we do not have score data for that ingredient
# counting the ingredients with no scores
count = 0
for key, value in ingred_score_dict.items():
    if value == 0:
        count +=1
count

70

In [261]:
# creating our new data frame with columns of interest
prod_ingred = sephora_data[new_cols + new_ingredients]
prod_ingred = prod_ingred.drop(["URL", "description", "full_description", "skin_type", "ingredients"], axis=1)
prod_ingred

,Label,brand,name,num_reviews,price,prod_size,rating,Combination,Dry,Normal,...,Limonium Vulgare Flower/Leaf/Stem Extract,Limonium Gerberi Extract,Lycopene,Limolool,Limnanthes alba (Meadowfoam) Seed Oil,Polyglyceryl-3 Rice Branate,Lime Tea Concentrate,Perfluoromethylcyclopentane,Oat Sugars,Pentylene Glycol / Ethylhexylglycerin
0,Moisturizers,Kiehl's Since 1851,Ultra Facial Cream,1132.0,32.0,1.70,4.3,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,BB & CC Creams,Dr. Jart+,Cicapair™ Tiger Grass Color Correcting Treatme...,3065.0,52.0,1.70,4.1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2,Moisturizers,Drunk Elephant,Protini™ Polypeptide Moisturizer,4872.0,68.0,1.69,4.0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
3,Moisturizers,La Mer,Crème de la Mer Moisturizer,695.0,190.0,1.00,3.9,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,Moisturizers,Tatcha,The Dewy Skin Cream Plumping & Hydrating Moist...,1727.0,68.0,1.70,4.2,0,1,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1133,Toners,Caudalie,Vinopure Natural Salicylic Acid Pore Minimizin...,325.0,28.0,6.70,4.0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
1134,Face Serums,OLEHENRIKSEN,Glow Cycle Retin-ALT Power Serum,1186.0,58.0,1.00,4.3,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1135,Face Masks,Summer Fridays,Overtime Mask,483.0,44.0,2.25,4.4,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1136,Face Masks,Caudalie,Instant Detox Mask,613.0,39.0,2.50,4.5,1,1,1,...,0,0,0,0,0,0,0,0,0,0


In [262]:
# populating scraped ingredient score information into prod_ingred df
# adds 10 new columns with counts of scores of each ingredient for each product
for col in prod_ingred.columns:
    # check if its an ingredient columns
    if col in ingred_score_dict:
        value = ingred_score_dict[col]
        col_name = "score{}_counts".format(str(value))
        
        if col_name not in prod_ingred.columns:
            prod_ingred[col_name] = prod_ingred[col]
        else:
            prod_ingred[col_name] += prod_ingred[col]

In [263]:
prod_ingred.head()

,Label,brand,name,num_reviews,price,prod_size,rating,Combination,Dry,Normal,...,score9_counts,score2_counts,score7_counts,score6_counts,score4_counts,score3_counts,score5_counts,score10_counts,score8_counts,score0_counts
0,Moisturizers,Kiehl's Since 1851,Ultra Facial Cream,1132.0,32.0,1.70,4.3,1,1,1,...,0,4,0,0,1,4,1,0,0,1
1,BB & CC Creams,Dr. Jart+,Cicapair™ Tiger Grass Color Correcting Treatme...,3065.0,52.0,1.70,4.1,1,1,1,...,0,12,0,0,0,6,1,0,0,0
2,Moisturizers,Drunk Elephant,Protini™ Polypeptide Moisturizer,4872.0,68.0,1.69,4.0,1,1,1,...,1,5,3,4,6,10,4,1,0,4
3,Moisturizers,La Mer,Crème de la Mer Moisturizer,695.0,190.0,1.00,3.9,0,1,0,...,0,6,0,1,1,4,3,0,1,0
4,Moisturizers,Tatcha,The Dewy Skin Cream Plumping & Hydrating Moist...,1727.0,68.0,1.70,4.2,0,1,1,...,1,9,4,3,6,9,3,1,1,0


In [326]:
# Build a recommendation system where you can toggle 2 options, product_type and skin_type

option_product_type = prod_ingred.Label.unique()
option_product_type = ['Eye Creams & Treatments', 'Face Masks', 'Moisturizers', 'Face Serums', 'Lip Balms & Treatments']
option_skin_type = ['Combination', 'Dry', 'Normal', 'Oily', 'Sensitive']

In [327]:
# old recommender had only one-hot encoded
numeric_columns = list(ingred_score_dict.keys()) 
def my_recommender(cosm_df, product_type_option, skin_type_option):
    """
    Given two options returns a dataframe with products matching the options and perform TSNE dimensionality reduction.
    """
    try:
        sub_matrix = cosm_df[cosm_df['Label'] == product_type_option][cosm_df[skin_type_option] == 1].copy()
        sub_matrix = sub_matrix.reset_index()
        model = TSNE(n_components = 2, learning_rate = 200)
        tsne_features = model.fit_transform(sub_matrix[numeric_columns])

        # Make X, Y columns
        sub_matrix['X'] = tsne_features[:, 0]
        sub_matrix['Y'] = tsne_features[:, 1]
        sub_matrix['pair'] = '{}_{}'.format(product_type_option, skin_type_option)
        return sub_matrix
    except:
        print('Failed for')
        print(product_type_option, skin_type_option)
        return None

_df = my_recommender(prod_ingred, 'Eye Creams & Treatments', 'Normal')

def update(op1 = option_product_type[0], op2 = option_skin_type[0]):
    a_b = op1 + '_' + op2
    new_data = {
        'X' : _df[_df['pair'] == a_b]['X'],
        'Y' : _df[_df['pair'] == a_b]['Y'],
        'name' : _df[_df['pair'] == a_b]['name'],
        'brand' : _df[_df['pair'] == a_b]['brand'],
        'price' : _df[_df['pair'] == a_b]['price'],
        'rating' : _df[_df['pair'] == a_b]['rating'],
    }
    source.data = new_data
    push_notebook() 

source = ColumnDataSource(_df)
plot = figure(x_axis_label = 'T-SNE 1',
              y_axis_label = 'T-SNE 2', 
              width = 500, height = 400)
plot.circle(x = 'X', y = 'Y', source = source, 
            size = 10, color = '#FF7373', alpha = .8)
plot.background_fill_color = "beige"
plot.background_fill_alpha = 0.2
# Add hover tool
hover = HoverTool(tooltips = [
        ('Item', '@name'),
        ('Brand', '@brand'),
        ('Price', '$ @price'),
        ('Rating', '@rating')])
plot.add_tools(hover)
output_notebook()
interact(update, op1=option_product_type, op2=option_skin_type)
show(plot, notebook_handle=True)

/Users/marissakhoury/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


Loading BokehJS ...

interactive(children=(Dropdown(description='op1', options=('Eye Creams & Treatments', 'Face Masks', 'Moisturiz…

In [274]:
from sklearn.manifold import TSNE

# TSNE

# use only numerical data for TSNE dimensionality reduction
numeric_columns = ["score{}_counts".format(str(i)) for i in range(1, 11)] + list(ingred_score_dict.keys())
design_matrix = prod_ingred[numeric_columns]
# plot in 2d
model = TSNE(n_components = 2, learning_rate = 200)
tsne_features = model.fit_transform(design_matrix)
prod_ingred['X'] = tsne_features[:, 0]
prod_ingred['Y'] = tsne_features[:, 1]
prod_ingred.head()

In [328]:
def my_recommender(cosm_df, product_type_option, skin_type_option):
    """
    Given two options returns a dataframe with products matching the options and perform TSNE dimensionality reduction.
    """
    try:
        sub_matrix = cosm_df[cosm_df['Label'] == product_type_option][cosm_df[skin_type_option] == 1].copy()
        sub_matrix = sub_matrix.reset_index()
        model = TSNE(n_components = 2, learning_rate = 200)
        tsne_features = model.fit_transform(sub_matrix[numeric_columns])

        # Make X, Y columns
        sub_matrix['X'] = tsne_features[:, 0]
        sub_matrix['Y'] = tsne_features[:, 1]
        sub_matrix['pair'] = '{}_{}'.format(product_type_option, skin_type_option)
        return sub_matrix
    except:
        print('Failed for')
        print(product_type_option, skin_type_option)
        return None

In [321]:
from sklearn.metrics.pairwise import cosine_similarity

from bokeh.io import show, curdoc, output_notebook, push_notebook
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, HoverTool, Select, Paragraph, TextInput
from bokeh.layouts import widgetbox, column, row
from ipywidgets import interact 



In [322]:
# Calculating the similarity of products which are moisturizers and for combination skin
my_recommender(prod_ingred, "Moisturizers", "Combination").head()

/Users/marissakhoury/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  This is separate from the ipykernel package so we can avoid doing imports until


,index,Label,brand,name,num_reviews,price,prod_size,rating,Combination,Dry,...,score6_counts,score4_counts,score3_counts,score5_counts,score10_counts,score8_counts,score0_counts,X,Y,pair
0,0,Moisturizers,Kiehl's Since 1851,Ultra Facial Cream,1132.0,32.0,1.70,4.3,1,1,...,0,1,4,1,0,0,1,21.151793,-18.021763,Moisturizers_Combination
1,2,Moisturizers,Drunk Elephant,Protini™ Polypeptide Moisturizer,4872.0,68.0,1.69,4.0,1,1,...,4,6,10,4,1,0,4,-19.715162,70.631310,Moisturizers_Combination
2,12,Moisturizers,La Mer,The Moisturizing Soft Cream,261.0,190.0,1.00,3.9,1,1,...,2,4,9,2,0,1,0,-38.675201,62.393147,Moisturizers_Combination
3,13,Moisturizers,Fresh,Rose & Hyaluronic Acid Deep Hydration Moisturizer,1546.0,42.0,1.60,4.2,1,1,...,0,2,7,1,0,1,1,11.898293,-26.579321,Moisturizers_Combination
4,14,Moisturizers,First Aid Beauty,Ultra Repair® Cream Intense Hydration,6910.0,14.0,2.00,4.6,1,1,...,0,1,1,1,0,0,1,44.808075,-29.832609,Moisturizers_Combination


In [323]:
# Build a dataframe for all combinations
df_all = pd.DataFrame()
for op_1 in option_product_type:
    for op_2 in option_skin_type:
        temp = my_recommender(prod_ingred, op_1, op_2)
        if temp is None:
            continue
        temp['Label'] = op_1 + '_' + op_2
        df_all = pd.concat([df_all, temp], ignore_index=True)

Eye Creams & Treatments Combination


/Users/marissakhoury/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  This is separate from the ipykernel package so we can avoid doing imports until


Eye Creams & Treatments Dry


/Users/marissakhoury/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  This is separate from the ipykernel package so we can avoid doing imports until


Eye Creams & Treatments Normal


...

Lip Balms & Treatments Normal


/Users/marissakhoury/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  This is separate from the ipykernel package so we can avoid doing imports until


Lip Balms & Treatments Oily


/Users/marissakhoury/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  This is separate from the ipykernel package so we can avoid doing imports until


Lip Balms & Treatments Sensitive
Failed for
Lip Balms & Treatments Sensitive


In [329]:
# Note that some combinations had no products, thus the empty graphs

_df = df_all

def update(op1 = option_product_type[0], op2 = option_skin_type[0]):
    a_b = op1 + '_' + op2
    new_data = {
        'X' : _df[_df['pair'] == a_b]['X'],
        'Y' : _df[_df['pair'] == a_b]['Y'],
        'name' : _df[_df['pair'] == a_b]['name'],
        'brand' : _df[_df['pair'] == a_b]['brand'],
        'price' : _df[_df['pair'] == a_b]['price'],
        'rating' : _df[_df['pair'] == a_b]['rating'],
    }
    source.data = new_data
    push_notebook() 

source = ColumnDataSource(_df)
plot = figure(x_axis_label = 'T-SNE 1',
              y_axis_label = 'T-SNE 2', 
              width = 500, height = 400)
plot.circle(x = 'X', y = 'Y', source = source, 
            size = 10, color = '#FF7373', alpha = .8)
plot.background_fill_color = "beige"
plot.background_fill_alpha = 0.2
# Add hover tool
hover = HoverTool(tooltips = [
        ('Item', '@name'),
        ('Brand', '@brand'),
        ('Price', '$ @price'),
        ('Rating', '@rating')])
plot.add_tools(hover)
output_notebook()
interact(update, op1=option_product_type, op2=option_skin_type)
show(plot, notebook_handle=True)

Loading BokehJS ...

interactive(children=(Dropdown(description='op1', options=('Eye Creams & Treatments', 'Face Masks', 'Moisturiz…

In [325]:
_df.head()

,index,Label,brand,name,num_reviews,price,prod_size,rating,Combination,Dry,...,score6_counts,score4_counts,score3_counts,score5_counts,score10_counts,score8_counts,score0_counts,X,Y,pair
0,297,Eye Creams & Treatments_Combination,Estée Lauder,Advanced Night Repair Concentrated Recovery Ey...,12.0,42.0,1.70,4.1,1,1,...,0,1,5,0,0,0,0,-10.241736,1.525178,Eye Creams & Treatments_Combination
1,323,Eye Creams & Treatments_Combination,Kiehl's Since 1851,Creamy Eye Treatment with Avocado,1030.0,32.0,0.50,4.0,1,1,...,2,2,3,0,0,0,0,-21.017664,12.545463,Eye Creams & Treatments_Combination
2,324,Eye Creams & Treatments_Combination,OLEHENRIKSEN,Banana Bright Eye Crème,3814.0,39.0,0.50,3.8,1,1,...,0,2,7,1,0,1,0,1.439093,-3.117286,Eye Creams & Treatments_Combination
3,325,Eye Creams & Treatments_Combination,Shiseido,Benefiance Wrinkle Smoothing Eye Cream,1270.0,70.0,0.51,4.4,1,1,...,1,5,8,3,0,1,1,-2.035480,5.767904,Eye Creams & Treatments_Combination
4,326,Eye Creams & Treatments_Combination,Youth To The People,Dream Eye Cream with Goji Stem Cell and Ceramides,172.0,48.0,0.50,4.6,1,1,...,3,7,6,5,1,0,0,20.199965,-2.009284,Eye Creams & Treatments_Combination
